In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [ ]:
data = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
data.head()

In [ ]:
cat_col = []
num_col = []
for i in data.columns:
    if data[i].value_counts().count() > 10:
        num_col.append(i)
    else:
        cat_col.append(i)
print(f'categorical columns: {cat_col}')
print(f'numerical columns: {num_col}')

In [ ]:
num_col = num_col[2:]
num_col

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,5))
j=0
for i in num_col:
    sns.histplot(data[i], ax=ax[j])
    j+=1
fig.suptitle('Histplot of Numerical Data')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,5))
j=0
for i in num_col:
    sns.boxplot(data[i], ax=ax[j], palette='cool')
    j+=1
fig.suptitle('Boxplot of Numerical Data')

In [ ]:
train = data.copy()

In [ ]:
bins = [-0.9, 1.0,
        1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0,
        2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 3]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
train['time_step'] = pd.cut(x=train['time_step'], bins=[-0.99, 0.5, 1, 1.5, 2, 2.5, 3], labels=[0,1,2,3,4,5])
train['time_step'] = train['time_step'].astype('int64')
train.head()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
sns.distplot(train['pressure'], ax=ax[0])

train['pressure'] = np.where(train['pressure'] < 0, 0, train['pressure'])
train['pressure'] = np.sqrt(np.sqrt(train['pressure']))

sns.distplot(train['pressure'], ax=ax[1])

In [ ]:
train_R = pd.get_dummies(train['R'], prefix='R')
train_C = pd.get_dummies(train['C'], prefix='C')
train_u_out = pd.get_dummies(train['u_out'], prefix='u_out')

In [ ]:
train = pd.concat([train, train_R, train_C, train_u_out], axis=1)
train = train.drop(['R', 'C', 'u_out'], axis=1)
train.head()

In [ ]:
def create_new_feat(df):
    df["u_in_sum"] = df.groupby("breath_id")["u_in"].transform("sum")
    df["u_in_cumsum"] = df.groupby("breath_id")["u_in"].cumsum()
    df["u_in_std"] = df.groupby("breath_id")["u_in"].transform("std")
    df["u_in_min"] = df.groupby("breath_id")["u_in"].transform("min")
    df["u_in_max"] = df.groupby("breath_id")["u_in"].transform("max")
    df["u_in_cumsum_reverse"] = df["u_in_sum"] - df["u_in_cumsum"]
    
    df["u_in_first"] = df.groupby("breath_id")["u_in"].transform("first")
    df["u_in_last"]  = df.groupby("breath_id")["u_in"].transform("last")
    
    df['time_diff']  = (df['time_step']).groupby(df['breath_id']).diff(1)
    df['time_diff2'] = (df['time_step']).groupby(df['breath_id']).diff(2)
    df['time_diff3'] = (df['time_step']).groupby(df['breath_id']).diff(3)
    df['time_diff4'] = (df['time_step']).groupby(df['breath_id']).diff(4)
    df['time_diff5'] = (df['time_step']).groupby(df['breath_id']).diff(5)
    df['time_diff6'] = (df['time_step']).groupby(df['breath_id']).diff(6)
    df['time_diff7'] = (df['time_step']).groupby(df['breath_id']).diff(7)
    df['time_diff8'] = (df['time_step']).groupby(df['breath_id']).diff(8)
    
#     df["u_in_lag1"] = df.groupby("breath_id")["u_in"].shift(1)
#     df["u_in_lead1"] = df.groupby("breath_id")["u_in"].shift(-1)
#     df["u_in_lag1_diff"] = df["u_in"] - df["u_in_lag1"]
#     df["u_in_lead1_diff"] = df["u_in"] - df["u_in_lead1"]
        
    df["time_passed"] = df.groupby("breath_id")["time_step"].diff()
    
    df['u_in_lag1']  = df.groupby('breath_id')['u_in'].shift(1)
    df['u_in_lag2']  = df.groupby('breath_id')['u_in'].shift(2)
    df['u_in_lag3']  = df.groupby('breath_id')['u_in'].shift(3)
    df['u_in_lag4']  = df.groupby('breath_id')['u_in'].shift(4)
    df['u_in_lag5']  = df.groupby('breath_id')['u_in'].shift(5)
    df['u_in_lag6']  = df.groupby('breath_id')['u_in'].shift(6)
    df['u_in_lag7']  = df.groupby('breath_id')['u_in'].shift(7)
    df['u_in_lag8']  = df.groupby('breath_id')['u_in'].shift(8)
    df['u_in_lag9']  = df.groupby('breath_id')['u_in'].shift(9)
    df['u_in_lag10'] = df.groupby('breath_id')['u_in'].shift(10)
    df['u_in_lag11'] = df.groupby('breath_id')['u_in'].shift(11)
    df['u_in_lag12'] = df.groupby('breath_id')['u_in'].shift(12)
    df['u_in_lag13'] = df.groupby('breath_id')['u_in'].shift(13)
    df['u_in_lag14'] = df.groupby('breath_id')['u_in'].shift(14)
    df['u_in_lag15'] = df.groupby('breath_id')['u_in'].shift(15)
    df['u_in_lag16'] = df.groupby('breath_id')['u_in'].shift(16)
    df['u_in_lag17'] = df.groupby('breath_id')['u_in'].shift(17)
    df['u_in_lag18'] = df.groupby('breath_id')['u_in'].shift(18)
    df['u_in_lag19'] = df.groupby('breath_id')['u_in'].shift(19)
    df['u_in_lag20'] = df.groupby('breath_id')['u_in'].shift(20)
    df['u_in_lag21'] = df.groupby('breath_id')['u_in'].shift(21)
    
    return df

In [ ]:
train = create_new_feat(train)
train = train.fillna(train.min())
train.head()

In [ ]:
test_data = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
test_data.head()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(18,5))
j=0
for i in num_col[:2]:
    sns.histplot(test_data[i], ax=ax[j])
    j+=1
fig.suptitle('Histplot of Numerical Data')

In [ ]:
test_data['time_step'] = pd.cut(x=test_data['time_step'], bins=bins, labels=labels)
test_data['time_step'] = test_data['time_step'].astype('int64')
test_data.head()

In [ ]:
test_data_R = pd.get_dummies(test_data['R'], prefix='R')
test_data_C = pd.get_dummies(test_data['C'], prefix='C')
test_data_u_out = pd.get_dummies(test_data['u_out'], prefix='u_out')

In [ ]:
test_data = pd.concat([test_data, test_data_R, test_data_C, test_data_u_out], axis=1)
test_data = test_data.drop(['R', 'C', 'u_out'], axis=1)
test_data.head()

In [ ]:
test_data = create_new_feat(test_data)
test_data = test_data.fillna(test_data.min())
test_data.head()

In [ ]:
X_train = train.drop('pressure', axis=1)
y_train = train['pressure']

In [ ]:
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': 7000,  # initially - 5000
    'learning_rate': 0.1,  #initially - 0.1
    'subsample': 0.95,
    'colsample_bytree': 0.11,
    'max_depth': 5,  #initially - 2
    'booster': 'gbtree', 
    'reg_lambda': 66.1,
    'reg_alpha': 15.9,
    'random_state':42,
    'tree_method':'gpu_hist',
    'gpu_id':0,
    'predictor':'gpu_predictor'
}

# xgb_params = {'learning_rate':0.1, 'n_estimators':10000, 'max_depth':9,
#                 'n_jobs':16, 'tree_method':'gpu_hist', 'gpu_id':0, 'predictor':'gpu_predictor'}

model = XGBRegressor(**xgb_params)

model.fit(X_train,y_train)

In [ ]:
pred = model.predict(test_data)

In [ ]:
sample = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
sample['id'] = test_data['id']
sample['pressure'] = pd.DataFrame({'pressure': pred[:]})
sample['pressure'] = np.square(np.square(sample['pressure']))
sample.head()


In [ ]:
q1 = sample['pressure'].quantile(0.001)
q2 = sample['pressure'].quantile(0.999)
sample['pressure'] = sample['pressure'].apply(lambda x: x if x>q1 else x*0.77)
sample['pressure'] = sample['pressure'].apply(lambda x: x if x<q2 else x*1.1)
sample.to_csv('submission_quantile.csv', index=False)